# Pipeline

In [40]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

### Simple example

In [41]:
# Load dataset
cancer = load_breast_cancer()
# Split data
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=42)
# Normalize train data
scaler = MinMaxScaler().fit(X_train)
X_train_scaler = scaler.transform(X_train)
# Normalize test data
X_test_scaler = scaler.transform(X_test)
# Model
svm = SVC()
svm.fit(X_train_scaler, y_train)
print(f"Train Score: {svm.score(X_train_scaler, y_train):.5f}")
print(f"Test Score:  {svm.score(X_test_scaler, y_test):.5f}")

Train Score: 0.98357
Test Score:  0.97902


### Fuite de données

In [42]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10],
              'gamma': [0.001, 0.01, 0.1, 1, 10]}

grid = GridSearchCV(SVC(), param_grid=param_grid, cv=5)
# <!> Big mistake : Nous nous sommes servis de tous les données d'apprentissage recalibrées pour la recherche sur grille
# Pour chaque division dans la validation croisée, une certaines partie du jeu de données d'apprentissage original 
# va être déclarée comme étant TRAIN set et une autre comme étant TEST set de cette division
# Dans ce cas --> Fuite d'information
# Pour éviter ce problème, la division du jeu de données au cours de la validation croisée 
# devrait être effectuée avant tout prétraitement --> Utiliser un pipeline
grid.fit(X_train_scaler, y_train)
print(f"Best cross validation accuracy : {grid.best_score_}")
print(f"Best param : {grid.best_params_}")
print(f"Test Accuracy score : {grid.score(X_test_scaler, y_test)}")

Best cross validation accuracy : 0.9788508891928865
Best param : {'C': 1, 'gamma': 1}
Test Accuracy score : 0.9790209790209791


## Pipeline

Pipeline pour exprimer le flux de travail d'apprentissage.

L'object Pipeline prend une liste d'étape et chaque étape est un tuple.

La classe _Pipeline_ n'est pas restreinte au prétraitement et à la classification.

Les conditiosn pour les estimateurs d'un pipeline sont:

- Toutes les étapes la dernière, doivent possèder les méthodes _fit_ et _transform_
    - _transform_ pour qu'ils soient capables de produire une nouvelle représentation des données qui pourra être utilisée dans l'étape suivante
- La derniere étape doit dispenser d'une fonction fit mais pas d'un de la fonction _predict_

Au cours de l'appel à Pipeline.fit, le pipeline appelle _fit_ puis _transform_ ou simplement _fit_transform_ sur chaque étape.

In [43]:
# Create pipeline
pipe = Pipeline([("scaler", MinMaxScaler()), 
                 ("svm", SVC())])

# Fit pipeline
pipe.fit(X_train, y_train)

# Score 
print(f"Test Score: {pipe.score(X_test, y_test):.5f}")

Test Score: 0.97902


In [56]:
pipe.steps

[('scaler', MinMaxScaler()), ('svm', SVC())]

## Pipeline dans des recherches sur grill

La syntaxe à employer pour définir une grille dans un pipeline consiste à spécifier :

Pour chaque paramètre **nom de l'étape __ non du paramètre exact concerné**.

_MinMaxScaler_ est réajusté uniquement pour les données d'apprentissage et donc aucune information ne fuite du jeu de test.



## Method 1 (long)

In [57]:
param_grid = {'svm__C': [0.001, 0.01, 0.1, 1],
              'svm__gamma': [0.001, 0.01, 0.1, 1]}

pipe = Pipeline([("scaler", MinMaxScaler()), ("svm", SVC())])
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)

grid.fit(X_train, y_train)

print(f"Best cross validation accuracy: {grid.best_score_:.5f}")
print(f"Test set score: {grid.score(X_test, y_test):.5f}")
print(f"Best param validation accuracy: {grid.best_params_}")

Best cross validation accuracy: 0.96944
Test set score: 0.97902
Best param validation accuracy: {'svm__C': 1, 'svm__gamma': 1}


## Method 2 (short)